In [1]:
import numpy as np 
import glob
import pandas as pd 
import xarray as xr
import rioxarray
dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/data/raw_data/gimms/Original/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/data/raw_data/gimms/Original_netcdf/"

def read_ibl(f):
    file = np.fromfile(f,dtype='>u2', count=-1)
    data_orig = file.reshape([2160,4320],order="F")
    data_orig = np.where(data_orig>20000,np.nan,data_orig)
    lai = np.floor(data_orig/10)*10*0.001
    qc = data_orig - np.floor(data_orig/10)*10
    return lai, qc

def xr_create(vars,t):
    lat_vec = np.flip(np.linspace(-90,90,2160))
    long_vec = np.linspace(-180,180,4320)
    ds = xr.Dataset(
        data_vars = dict(
            lai = (["lat","long"],vars[0]),
            qc = (["lat","long"],vars[1])),
        coords = dict(
            lat=(["lat"],lat_vec),
            long=(["long"],long_vec),
            # time = np.datetime64(t))
            ))
    ds = ds.expand_dims({"time":[np.datetime64(t)]})
    ds.rio.write_crs(
        4326,
        inplace=True,
    ).rio.set_spatial_dims(
        x_dim="long",
        y_dim="lat",
        inplace=True,
    ).rio.write_coordinate_system(inplace=True)
    return ds 


In [2]:

years = np.arange(1982,2016)
months = ["01","02","03","04","05","06","07","08","09","10","11","12"]
months_name = ["jan", "feb","mar","apr","may","jun","jul","aug",
              "sep","oct","nov","dec"]

for i in range(len(years)):
    for j in range(len(months)):
        date_a = np.datetime64(str(years[i])+"-"+str(months[j])+"-01")
        date_b = np.datetime64(str(years[i])+"-"+str(months[j])+"-15")
        
        print(date_a)

        fname_a = dir+"AVHRRBUVI04."+str(years[i])+months_name[j]+"a.abl"
        fname_b = dir+"AVHRRBUVI04."+str(years[i])+months_name[j]+"b.abl"

        vars_a = read_ibl(fname_a)
        vars_b = read_ibl(fname_b)

        ds_a = xr_create(vars_a,date_a)
        ds_b = xr_create(vars_b,date_b)

        ds_a.to_netcdf(out_dir+"AVHRRBUVI04."+str(years[i])+months_name[j]+"a.nc")
        ds_b.to_netcdf(out_dir+"AVHRRBUVI04."+str(years[i])+months_name[j]+"b.nc")



1982-01-01
1982-02-01
1982-03-01
1982-04-01


In [1]:
gimms_lai = xr.open_mfdataset(out_dir+'*.nc',concat_dim=['time'], combine='nested').sortby("time")


NameError: name 'xr' is not defined